# VAE를 이용한 생성 모델
- 새로운 분자의 SMILES를 출력
- MolrculeNet이 제공하는 SMILES 데이터셋 MUV 사용 (약 90000개 제공)
 - Maximum Unbiased Validation(MUV) - 17개의 태스크 포함

In [ ]:
!pip install DeepChem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 30.4 MB/s 
     |████████████████████████████████| 29.5 MB 1.3 MB/s 


In [ ]:
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

import pandas as pd
import numpy as np
%config InlineBackend.figure_format = 'retina'

In [ ]:
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids

# SMILES 문자열의 규칙을 파악: 문자(토큰)의 목록, 문자열의 최대길이 등

tokens = set()
for s in train_smiles:
  tokens = tokens.union(set(s))
tokens = sorted(list(tokens))
max_length = max(len(s) for s in train_smiles)

# 모델 만들기
# AspuruGuzikAutoEncoder: 인코더는 합성곱신경망을, 디코더는 순환신경망을 사용
# 학습속도를 조절하기 위해서 ExponentialDecay를 사용한다
# 0.001에서 시작하고 이포크마다 0.95배로 감소시킨다

from deepchem.models.optimizers import ExponentialDecay
from deepchem.models.seqtoseq import AspuruGuzikAutoEncoder
batch_size = 100
batches_per_epoch = len(train_smiles)/batch_size
learning_rate = ExponentialDecay(0.001, 0.95, batches_per_epoch)
model = AspuruGuzikAutoEncoder(tokens, max_length, model_dir='vae', batch_size=batch_size, learning_rate=learning_rate)

# 시퀀스 생성 함수 정의

def generate_sequences(epochs):
  for i in range(epochs):
    for s in train_smiles:
      yield (s, s)

# AspuruGuzikAutoEncoder이 제공하는 자체 학습 함수 (이포크수 지정)

model.fit_sequences(generate_sequences(50)) # 50 이포크 수


In [ ]:
# 학습된 모델을 이용하여 새로운 분자를 만든다
# 모델에 들어가는 벡터의 크기를 지정한다 (예: 196)
# 벡터를 1000개 생성하겠다
# 생성된 분자들중 유효한 SMILES를 걸러내기 위해서 RDKit의 MolFromSmiles를 사용한다

from rdkit import Chem
predictions = model.predict_from_embeddings(np.random.normal(size=(1000,196)))
molecules = []
for p in predictions:
  smiles = ''.join(p)
  if Chem.MolFromSmiles(smiles) is not None:
    molecules.append(smiles)
print()
print('Generated molecules:')
for m in molecules:
  print(m)


Generated molecules:
CC1Cn2c(CCCCCCCCCCCCCC)c(C#CCO)c2c1
CC1CC1NCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
c1ccc(CN2CCCCCCCCCCC3)c(=C)cc3cccc21
CC1c(NCC(=O)CC)cc(C)ccnccccccc2ccccccc21
Cc1ncccc(CCCCCCCCCCCCCC)cc(CO)cc1


In [10]:
molecules

['CC1Cn2c(CCCCCCCCCCCCCC)c(C#CCO)c2c1',
 'CC1CC1NCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC',
 'c1ccc(CN2CCCCCCCCCCC3)c(=C)cc3cccc21',
 'CC1c(NCC(=O)CC)cc(C)ccnccccccc2ccccccc21',
 'Cc1ncccc(CCCCCCCCCCCCCC)cc(CO)cc1']

## 생성된 SMILES 들에 대해서 유효하지 않거나 약물로서 가치가 없는 분자를 걸러내야 한다
- 제거하고 싶은 분자가 있는지 찾는다
- MolFromSmiles()을 사용해 SMILES 문자열들을 분자 객체로 변환한다
- 분자의 크기를 확인한다 (10보다 작으면 상호작용에 필요한 에너지가 불충분하고, 50 이상이면 분자의 용해도가 너무 낮아 문제가 된다)
- 수소를 제외한 분자의 크기를 GetNumAtoms()로 얻는다
- 약물과 얼마나 유사한지를 판단하기 위해서 QED(Quantitave Estimate of Drugness)를 많이 사용한다
 - QED: 계산된 속성 집합과 판매된 약물의 동일한 특성 분포를 정량화 한 것 (Richard Bickerton 이 제안)
 - 1에 가까울수록 기존의 약물과 유사하다고 본다
 - QED > 0.5 인 분자만 고른 후 결과를 시각화 한다

In [11]:
from rdkit import Chem
molecules_new = [Chem.MolFromSmiles(x) for x in molecules]
# moleculs_new = [Chem.MolFromSmiles(x) for x in smlies_list]
print(sorted(x.GetNumAtoms() for x in molecules_new))

[25, 26, 27, 29, 79]


In [12]:
good_mol_list = [x for x in molecules_new if x.GetNumAtoms() > 10 and x.GetNumAtoms() < 50]
print(len(good_mol_list))

4


In [14]:
from rdkit.Chem import QED
qed_list = [QED.qed(x) for x in good_mol_list]

In [15]:
qed_list

[0.31427961330252857,
 0.4796533123360689,
 0.6721386351907204,
 0.34927910983033633]

In [16]:
final_mol_list = [(a,b) for a,b in zip(good_mol_list, qed_list) if b > 0.3] # 0.5를 낮춤
final_mol_list

[(<rdkit.Chem.rdchem.Mol at 0x7fac575196c0>, 0.31427961330252857),
 (<rdkit.Chem.rdchem.Mol at 0x7fab50189030>, 0.4796533123360689),
 (<rdkit.Chem.rdchem.Mol at 0x7fab50189df0>, 0.6721386351907204),
 (<rdkit.Chem.rdchem.Mol at 0x7fab50189850>, 0.34927910983033633)]

In [17]:
from rdkit.Chem import Draw
Draw.MolsToGridImage(
    [x[0] for x  in final_mol_list],
    molsPerRow = 3, useSVG=True,
    subImgSize = (250, 250),
    returnPNG=True,
    legends=[f"{x[1]:.2f}" for x in final_mol_list])


"<?xml version='1.0' encoding='iso-8859-1'?>\n<svg version='1.1' baseProfile='full'\n              xmlns='http://www.w3.org/2000/svg'\n                      xmlns:rdkit='http://www.rdkit.org/xml'\n                      xmlns:xlink='http://www.w3.org/1999/xlink'\n                  xml:space='preserve'\nwidth='750px' height='500px' viewBox='0 0 750 500'>\n<!-- END OF HEADER -->\n<rect style='opacity:1.0;fill:#FFFFFF;stroke:none' width='750.0' height='500.0' x='0.0' y='0.0'> </rect>\n<path class='bond-0 atom-0 atom-1' d='M 35.5,163.6 L 39.4,149.9' style='fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-1 atom-1 atom-2' d='M 39.4,149.9 L 52.7,144.9' style='fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2.0px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1' />\n<path class='bond-2 atom-2 atom-3' d='M 52.7,144.9 L 52.5,139.1' style='fill:none;fill-rule:evenodd;stroke:#000000;str

참고:
https://www.rdkit.org/docs/GettingStartedInPython.html#drawing-molecules

In [18]:
img=Draw.MolsToGridImage(subms,molsPerRow=4,subImgSize=(200,200),
                         legends=[x.GetProp("_Name") for x in subms])    
img.save('images/cdk2_molgrid.aligned.o.png')

NameError: ignored

In [ ]:
final_mol_list

[]